# UniFi API Configuration

This notebook helps you configure API access for both **UniFi Site Manager API** (cloud/remote) and **UniFi Network Application API** (local controller).

All credentials will be stored in `~/.env` for security and reuse across notebooks.

---

## Architecture Overview

```mermaid
graph TB
    subgraph "Your Environment"
        Notebook["📓 Jupyter Notebook<br/>This Configuration Tool"]
        EnvFile["🔐 ~/.env<br/>Credential Storage"]
    end
    
    subgraph "Remote/Cloud API"
        SiteManager["☁️ UniFi Site Manager<br/>https://unifi.ui.com"]
        SiteAPI["🌐 Site Manager API<br/>https://api.ui.com/v1/*"]
        APIKey["🔑 API Key<br/>Read-Only Access"]
    end
    
    subgraph "Local Network"
        LocalController["🏠 Local UniFi Controller<br/>https://192.168.1.1"]
        LocalAPI["🔧 Network Application API<br/>/proxy/network/api/*"]
        LocalToken["🔑 API Token<br/>Read/Write Access"]
        UserPass["👤 Username/Password<br/>Optional Discovery"]
    end
    
    Notebook -->|"Store Credentials"| EnvFile
    EnvFile -->|"UNIFI_API_KEY"| APIKey
    EnvFile -->|"UNIFI_LOCAL_TOKEN"| LocalToken
    EnvFile -->|"UNIFI_USERNAME<br/>UNIFI_PASSWORD"| UserPass
    
    APIKey -->|"Authenticate"| SiteAPI
    SiteManager -->|"Generate"| APIKey
    
    LocalToken -->|"Authenticate"| LocalAPI
    UserPass -.->|"Optional"| LocalAPI
    LocalController -->|"Generate"| LocalToken
    
    SiteAPI -->|"Read-Only<br/>Monitoring & Analytics"| Notebook
    LocalAPI -->|"Read/Write<br/>Full Configuration"| Notebook
    
    style Notebook fill:#e1f5ff
    style EnvFile fill:#fff4e1
    style SiteAPI fill:#e8f5e9
    style LocalAPI fill:#fce4ec
```

---

## Prerequisites

- Access to UniFi Site Manager at https://unifi.ui.com
- Access to local UniFi controller (typically https://192.168.1.1)
- 2FA enabled on your UniFi account (for API token creation)


In [ ]:
# Install required libraries
import sys
!{sys.executable} -m pip install requests python-dotenv matplotlib --quiet

print("✓ Libraries installed")


In [ ]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv, set_key

# Path to .env file in home directory
env_path = Path.home() / '.env'

# Load existing .env if it exists
if env_path.exists():
    load_dotenv(env_path)
    print(f"✓ Loaded existing ~/.env")
else:
    print(f"ℹ Creating new ~/.env file")

print(f"✓ Environment file: {env_path}")


## Step 1: Site Manager API Key (Remote/Cloud Access)

The Site Manager API provides read-only access to monitor your UniFi deployments from the cloud.

### How to Get Your API Key

1. **Sign in** to UniFi Site Manager at [https://unifi.ui.com](https://unifi.ui.com)
2. **Navigate** to the API section from the left navigation bar
3. **Click** "Create API Key"
4. **Copy** the generated key immediately (it's only shown once!)
5. **Store** it securely

### API Key Capabilities

- ✅ List hosts, sites, and devices
- ✅ Get device status and analytics
- ✅ Query ISP metrics
- ✅ View SD-WAN configurations (read-only)
- ❌ No write operations (read-only API)

**Documentation**: [https://developer.ui.com/site-manager-api/gettingstarted](https://developer.ui.com/site-manager-api/gettingstarted)


In [ ]:
# Visualize API comparison
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Title
ax.text(5, 9.5, 'UniFi API Comparison', ha='center', va='top', fontsize=20, fontweight='bold')

# Site Manager API Box
sm_box = FancyBboxPatch((0.5, 5.5), 4, 3.5, boxstyle="round,pad=0.1", 
                        edgecolor='#2196F3', facecolor='#E3F2FD', linewidth=2)
ax.add_patch(sm_box)
ax.text(2.5, 8.5, 'Site Manager API', ha='center', va='top', fontsize=16, fontweight='bold')
ax.text(2.5, 7.8, '(Remote/Cloud)', ha='center', va='top', fontsize=12, style='italic')

# Read-only badge
ro_badge = FancyBboxPatch((1.5, 7.2), 2, 0.5, boxstyle="round,pad=0.05",
                          edgecolor='#4CAF50', facecolor='#C8E6C9', linewidth=1.5)
ax.add_patch(ro_badge)
ax.text(2.5, 7.45, 'READ-ONLY', ha='center', va='center', fontsize=10, fontweight='bold', color='#2E7D32')

# Capabilities
sm_capabilities = [
    '✓ List hosts, sites, devices',
    '✓ Device status & analytics',
    '✓ ISP metrics',
    '✓ SD-WAN configs (view)',
    '✗ No write operations'
]

for i, cap in enumerate(sm_capabilities):
    color = '#2E7D32' if cap.startswith('✓') else '#D32F2F'
    ax.text(2.5, 6.8 - i*0.35, cap, ha='center', va='top', fontsize=10, color=color)

# Local Network API Box
local_box = FancyBboxPatch((5.5, 5.5), 4, 3.5, boxstyle="round,pad=0.1",
                           edgecolor='#E91E63', facecolor='#FCE4EC', linewidth=2)
ax.add_patch(local_box)
ax.text(7.5, 8.5, 'Network Application API', ha='center', va='top', fontsize=16, fontweight='bold')
ax.text(7.5, 7.8, '(Local Controller)', ha='center', va='top', fontsize=12, style='italic')

# Read-write badge
rw_badge = FancyBboxPatch((6.5, 7.2), 2, 0.5, boxstyle="round,pad=0.05",
                         edgecolor='#E91E63', facecolor='#F8BBD0', linewidth=1.5)
ax.add_patch(rw_badge)
ax.text(7.5, 7.45, 'READ/WRITE', ha='center', va='center', fontsize=10, fontweight='bold', color='#AD1457')

# Capabilities
local_capabilities = [
    '✓ DNS configuration',
    '✓ VPN tunnels',
    '✓ Routing & WAN failover',
    '✓ Firewall rules',
    '✓ Full network settings'
]

for i, cap in enumerate(local_capabilities):
    ax.text(7.5, 6.8 - i*0.35, cap, ha='center', va='top', fontsize=10, color='#2E7D32')

# Authentication section
auth_box = FancyBboxPatch((1, 0.5), 8, 4, boxstyle="round,pad=0.1",
                          edgecolor='#FF9800', facecolor='#FFF3E0', linewidth=2)
ax.add_patch(auth_box)
ax.text(5, 4.2, 'Authentication Methods', ha='center', va='top', fontsize=16, fontweight='bold')

# Site Manager auth
ax.text(2.5, 3.7, 'Site Manager API', ha='center', va='top', fontsize=12, fontweight='bold')
ax.text(2.5, 3.3, '🔑 API Key', ha='center', va='top', fontsize=11)
ax.text(2.5, 2.9, 'From: unifi.ui.com', ha='center', va='top', fontsize=9, style='italic')
ax.text(2.5, 2.5, 'No 2FA required', ha='center', va='top', fontsize=9, color='#4CAF50')

# Local API auth
ax.text(7.5, 3.7, 'Network Application API', ha='center', va='top', fontsize=12, fontweight='bold')
ax.text(7.5, 3.3, '🔑 API Token', ha='center', va='top', fontsize=11)
ax.text(7.5, 2.9, 'From: Local Controller', ha='center', va='top', fontsize=9, style='italic')
ax.text(7.5, 2.5, '⚠️ 2FA required', ha='center', va='top', fontsize=9, color='#FF9800')

# Optional credentials
ax.text(5, 2, 'Optional: Username/Password', ha='center', va='top', fontsize=10, style='italic')
ax.text(5, 1.6, '(For discovery operations)', ha='center', va='top', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# Check if API key already exists
existing_key = os.getenv('UNIFI_API_KEY')

if existing_key:
    print(f"✓ UNIFI_API_KEY already set (first 10 chars: {existing_key[:10]}...)")
    print("\nTo update, enter a new key below, or set UNIFI_API_KEY='' to clear.")
else:
    print("ℹ UNIFI_API_KEY not found in ~/.env")

print("\n" + "="*60)
print("ENTER SITE MANAGER API KEY")
print("="*60)
print("\nIf you need to get a key:")
print("1. Go to https://unifi.ui.com")
print("2. Navigate to API section → Create API Key")
print("3. Copy the key (shown only once!)")
print("\nEnter your API key below (or press Enter to skip/keep existing):")

# In interactive mode, you would use input(), but for notebook we'll check env
# You can set it directly in the next cell or modify ~/.env manually
print("\n💡 Option 1: Enter key in the next cell")
print("💡 Option 2: Manually edit ~/.env and add: UNIFI_API_KEY=your_key_here")
print("\nWaiting for input... (you'll need to run the next cell to set it)")


In [ ]:
# Set or update API key
# MODIFY THIS CELL: Replace 'YOUR_API_KEY_HERE' with your actual API key, or leave empty to keep existing

new_api_key = 'YOUR_API_KEY_HERE'  # <-- REPLACE THIS with your actual API key

if new_api_key and new_api_key != 'YOUR_API_KEY_HERE':
    # Store in .env file
    set_key(env_path, 'UNIFI_API_KEY', new_api_key)
    print(f"✓ API key stored in ~/.env")
    print(f"✓ First 10 characters: {new_api_key[:10]}...")
    
    # Reload environment
    load_dotenv(env_path, override=True)
else:
    existing = os.getenv('UNIFI_API_KEY')
    if existing:
        print(f"✓ Using existing API key (first 10 chars: {existing[:10]}...)")
    else:
        print("⚠️  No API key set. Please set new_api_key in this cell or add to ~/.env manually")
        print("   Format: UNIFI_API_KEY=your_key_here")


In [ ]:
# Test Site Manager API connection
print("="*60)
print("TESTING SITE MANAGER API")
print("="*60)

api_key = os.getenv('UNIFI_API_KEY')

if not api_key:
    print("✗ UNIFI_API_KEY not found. Please set it in the previous cell or ~/.env")
else:
    api_session = requests.Session()
    api_session.headers.update({
        'X-API-Key': api_key,
        'Accept': 'application/json',
    })
    
    # Test with hosts endpoint
    try:
        resp = api_session.get('https://api.ui.com/v1/hosts', timeout=15)
        
        if resp.status_code == 200:
            data = resp.json()
            if 'data' in data:
                host_count = len(data['data']) if isinstance(data['data'], list) else 0
                print(f"✓ Connection successful!")
                print(f"✓ Found {host_count} host(s)")
                print(f"✓ API key is valid and working")
            else:
                print(f"✓ Connection successful (unexpected response format)")
        elif resp.status_code == 401:
            print(f"✗ Authentication failed - API key is invalid")
            print(f"   Please check your API key and try again")
        elif resp.status_code == 403:
            print(f"✗ Forbidden - API key may not have required permissions")
        else:
            print(f"✗ Error {resp.status_code}: {resp.text[:100]}")
    except requests.exceptions.RequestException as e:
        print(f"✗ Connection error: {e}")
        print(f"   Check your internet connection")


## Step 2: Local Network Application API Token

The local API provides **full read/write access** for configuration management (DNS, VPN, routing, firewall rules, etc.).

### How to Create an API Token

1. **Log in** to your local UniFi controller:
   - Typically: `https://192.168.1.1` or `https://your-controller-ip:443`
2. **Navigate** to Settings (gear icon) or User Settings
3. **Find** "API Tokens" section
4. **Click** "Create New Token" or "Generate Token"
5. **Configure** the token:
   - Name: Give it a descriptive name (e.g., "Notebook Access", "Python Script")
   - Permissions: Select "Network" management permissions
   - Expiration: Set expiration date (or leave as "Never" for long-term use)
6. **Copy** the token immediately (shown only once!)
7. **Store** it securely

### Important Notes

- ⚠️ **2FA Required**: You must have 2FA enabled on your UniFi account to create API tokens
- 🔒 **Security**: Treat API tokens like passwords - never share them or commit to git
- 🔄 **Token Expiration**: If you set an expiration date, you'll need to regenerate the token
- 📋 **Permissions**: Ensure the token has "Network" management permissions for full access

### API Token Capabilities

- ✅ Read network configurations
- ✅ Modify DNS settings
- ✅ Create/modify VPN tunnels
- ✅ Configure routing and WAN failover
- ✅ Manage firewall rules
- ✅ Full read/write access to all network settings


In [ ]:
# Check if local API token already exists
existing_token = os.getenv('UNIFI_LOCAL_TOKEN')

if existing_token:
    print(f"✓ UNIFI_LOCAL_TOKEN already set (first 10 chars: {existing_token[:10]}...)")
    print("\nTo update, enter a new token below, or set UNIFI_LOCAL_TOKEN='' to clear.")
else:
    print("ℹ UNIFI_LOCAL_TOKEN not found in ~/.env")

print("\n" + "="*60)
print("ENTER LOCAL API TOKEN")
print("="*60)
print("\nIf you need to create a token:")
print("1. Go to https://192.168.1.1 (or your controller IP)")
print("2. Settings → API Tokens → Create New Token")
print("3. Set permissions (Network management)")
print("4. Copy the token (shown only once!)")
print("\nEnter your API token below (or press Enter to skip/keep existing):")

print("\n💡 Option 1: Enter token in the next cell")
print("💡 Option 2: Manually edit ~/.env and add: UNIFI_LOCAL_TOKEN=your_token_here")
print("\nWaiting for input... (you'll need to run the next cell to set it)")


In [ ]:
# Set or update local API token
# MODIFY THIS CELL: Replace 'YOUR_API_TOKEN_HERE' with your actual API token, or leave empty to keep existing

new_local_token = 'YOUR_API_TOKEN_HERE'  # <-- REPLACE THIS with your actual API token

if new_local_token and new_local_token != 'YOUR_API_TOKEN_HERE':
    # Store in .env file
    set_key(env_path, 'UNIFI_LOCAL_TOKEN', new_local_token)
    print(f"✓ Local API token stored in ~/.env")
    print(f"✓ First 10 characters: {new_local_token[:10]}...")
    
    # Reload environment
    load_dotenv(env_path, override=True)
else:
    existing = os.getenv('UNIFI_LOCAL_TOKEN')
    if existing:
        print(f"✓ Using existing local API token (first 10 chars: {existing[:10]}...)")
    else:
        print("⚠️  No local API token set. Please set new_local_token in this cell or add to ~/.env manually")
        print("   Format: UNIFI_LOCAL_TOKEN=your_token_here")


## Step 3: Local Discovery Credentials (Optional)

For some discovery operations and device/client enumeration, you may need local controller username/password credentials.

### When These Are Used

- Device and client discovery from local controllers
- Historical data retrieval
- Some legacy authentication methods
- **Note**: Modern UniFi OS with 2FA typically requires API tokens instead

### Important Security Note

⚠️ **Storing passwords in plain text**: While convenient, storing passwords in `~/.env` is less secure than API tokens. Consider using API tokens for production use.

### Username

Typically your UniFi account email or username used to log into the UniFi controller.


In [ ]:
# Check if username already exists
existing_username = os.getenv('UNIFI_USERNAME')

if existing_username:
    print(f"✓ UNIFI_USERNAME already set: {existing_username}")
    print("\nTo update, enter a new username below, or set UNIFI_USERNAME='' to clear.")
else:
    print("ℹ UNIFI_USERNAME not found in ~/.env")

print("\n" + "="*60)
print("ENTER LOCAL CONTROLLER USERNAME")
print("="*60)
print("\nThis is typically your UniFi account email or username.")
print("Example: user@example.com or admin")
print("\nEnter your username below (or press Enter to skip/keep existing):")

print("\n💡 Option 1: Enter username in the next cell")
print("💡 Option 2: Manually edit ~/.env and add: UNIFI_USERNAME=your_username_here")


In [ ]:
# Final Configuration Status Visualization
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path.home() / '.env'
load_dotenv(env_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Credential Status
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.text(5, 9.5, 'Configuration Status', ha='center', fontsize=18, fontweight='bold')

# Check each credential
credentials = [
    ('UNIFI_API_KEY', 'Site Manager API Key', '#2196F3'),
    ('UNIFI_LOCAL_TOKEN', 'Local API Token', '#E91E63'),
    ('UNIFI_USERNAME', 'Username (Optional)', '#FF9800'),
    ('UNIFI_PASSWORD', 'Password (Optional)', '#FF9800'),
]

y_start = 8
for i, (key, label, color) in enumerate(credentials):
    y_pos = y_start - i * 1.8
    
    # Check if set
    value = os.getenv(key)
    is_set = value is not None and value != '' and value != f'YOUR_{key}_HERE'
    
    # Status box
    status_color = '#C8E6C9' if is_set else '#FFCDD2'
    status_text = '✓ Configured' if is_set else '✗ Not Set'
    status_box = FancyBboxPatch((0.5, y_pos - 0.4), 3, 0.7, 
                               boxstyle="round,pad=0.05", edgecolor=status_color, 
                               facecolor=status_color, linewidth=1.5)
    ax1.add_patch(status_box)
    ax1.text(2, y_pos, status_text, ha='center', va='center', 
            fontsize=11, fontweight='bold', color='#2E7D32' if is_set else '#D32F2F')
    
    # Label
    ax1.text(4.5, y_pos, label, ha='left', va='center', fontsize=11)
    
    # Preview if set
    if is_set and key != 'UNIFI_PASSWORD':
        preview = value[:15] + '...' if len(value) > 15 else value
        ax1.text(8, y_pos, preview, ha='right', va='center', 
                fontsize=9, style='italic', color='#666')
    elif is_set and key == 'UNIFI_PASSWORD':
        ax1.text(8, y_pos, '••••••••', ha='right', va='center', 
                fontsize=9, style='italic', color='#666')

ax1.text(5, 1, 'All credentials stored in ~/.env', ha='center', 
        fontsize=10, style='italic', color='#666')

# Right: Credential Storage Structure
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.text(5, 9.5, 'Credential Storage (~/.env)', ha='center', fontsize=18, fontweight='bold')

# File structure
file_box = FancyBboxPatch((1, 1), 8, 7.5, boxstyle="round,pad=0.2",
                          edgecolor='#9C27B0', facecolor='#F3E5F5', linewidth=2)
ax2.add_patch(file_box)

ax2.text(5, 8, '~/.env', ha='center', va='top', fontsize=14, fontweight='bold')

# Show structure
env_structure = [
    'UNIFI_API_KEY=your_api_key_here',
    'UNIFI_LOCAL_TOKEN=your_token_here',
    'UNIFI_USERNAME=user@example.com',
    'UNIFI_PASSWORD=your_password',
]

y_start = 7
for i, line in enumerate(env_structure):
    y_pos = y_start - i * 1.2
    # Show line number
    ax2.text(1.5, y_pos, f'{i+1}', ha='center', va='center', 
            fontsize=9, color='#666')
    # Show content
    ax2.text(5, y_pos, line, ha='center', va='center', 
            fontsize=10, family='monospace', color='#333')

ax2.text(5, 2, 'Format: KEY=value', ha='center', fontsize=9, 
        style='italic', color='#666')
ax2.text(5, 1.5, 'One key per line', ha='center', fontsize=9, 
        style='italic', color='#666')

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*60)
print("CONFIGURATION SUMMARY")
print("="*60)
for key, label, _ in credentials:
    value = os.getenv(key)
    is_set = value is not None and value != '' and value != f'YOUR_{key}_HERE'
    status = "✓ Configured" if is_set else "✗ Not Set"
    print(f"{status:15s} - {label}")
print("\n" + "="*60)


In [ ]:
# Visualize authentication flow
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch, Circle
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left: Site Manager API Flow
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.text(5, 9.5, 'Site Manager API (Remote)', ha='center', fontsize=14, fontweight='bold')

# Components
components1 = [
    (2, 7.5, 'User', '#2196F3'),
    (5, 7.5, 'unifi.ui.com', '#4CAF50'),
    (8, 7.5, 'API Key', '#FF9800'),
    (5, 4.5, 'api.ui.com', '#E91E63'),
    (5, 2, '~/.env', '#9C27B0'),
]

for x, y, label, color in components1:
    box = FancyBboxPatch((x-1, y-0.5), 2, 0.8, boxstyle="round,pad=0.1",
                        edgecolor=color, facecolor='white', linewidth=2)
    ax1.add_patch(box)
    ax1.text(x, y, label, ha='center', va='center', fontsize=10, fontweight='bold')

# Arrows
arrows1 = [
    ((3, 7.5), (4, 7.5), 'Login'),
    ((6, 7.5), (7, 7.5), 'Generate'),
    ((8, 7), (5, 5), 'Use Key'),
    ((5, 4.2), (5, 2.8), 'Store'),
]

for (x1, y1), (x2, y2), label in arrows1:
    arrow = FancyArrowPatch((x1, y1), (x2, y2),
                           arrowstyle='->', lw=2, color='#666',
                           connectionstyle='arc3,rad=0.1')
    ax1.add_patch(arrow)
    mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
    ax1.text(mid_x, mid_y + 0.3, label, ha='center', fontsize=8, 
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Right: Local API Flow
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.text(5, 9.5, 'Network Application API (Local)', ha='center', fontsize=14, fontweight='bold')

# Components with 2FA
components2 = [
    (2, 7.5, 'User', '#2196F3'),
    (2, 6, '2FA', '#FF5722'),
    (5, 7.5, 'Local Controller', '#4CAF50'),
    (8, 7.5, 'API Token', '#FF9800'),
    (5, 4.5, 'Local API', '#E91E63'),
    (5, 2, '~/.env', '#9C27B0'),
]

for x, y, label, color in components2:
    box = FancyBboxPatch((x-1, y-0.5), 2, 0.8, boxstyle="round,pad=0.1",
                        edgecolor=color, facecolor='white', linewidth=2)
    ax2.add_patch(box)
    ax2.text(x, y, label, ha='center', va='center', fontsize=10, fontweight='bold')

# 2FA connection
arrow_2fa = FancyArrowPatch((2, 7), (2, 6.5),
                           arrowstyle='->', lw=2, color='#FF5722')
ax2.add_patch(arrow_2fa)
ax2.text(1.3, 6.75, 'Required', ha='center', fontsize=8, color='#FF5722', fontweight='bold')

# Arrows
arrows2 = [
    ((3, 7.5), (4, 7.5), 'Login'),
    ((6, 7.5), (7, 7.5), 'Create'),
    ((8, 7), (5, 5), 'Use Token'),
    ((5, 4.2), (5, 2.8), 'Store'),
]

for (x1, y1), (x2, y2), label in arrows2:
    arrow = FancyArrowPatch((x1, y1), (x2, y2),
                           arrowstyle='->', lw=2, color='#666',
                           connectionstyle='arc3,rad=0.1')
    ax2.add_patch(arrow)
    mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
    ax2.text(mid_x, mid_y + 0.3, label, ha='center', fontsize=8,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()


## Debugging: Launch Notebook in Browser

**For better visualization viewing**, launch this notebook in a browser with Playwright automation hooks!


In [ ]:
# Launch this notebook in browser for better visualization & debugging
from playwright.sync_api import sync_playwright
import subprocess
import time
import sys
from pathlib import Path

notebook_file = Path("unifi_api_configuration.ipynb")
notebook_name = notebook_file.name

# Check for Jupyter server
jupyter_url = None
try:
    import requests
    for port in [8888, 8889, 8890]:
        try:
            response = requests.get(f'http://localhost:{port}', timeout=2)
            if response.status_code == 200:
                jupyter_url = f"http://localhost:{port}"
                break
        except:
            pass
except:
    pass

if not jupyter_url:
    print("🚀 Starting Jupyter server...")
    subprocess.Popen([sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888'],
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(5)
    jupyter_url = "http://localhost:8888"

notebook_url = f"{jupyter_url}/notebooks/{notebook_name}"

print(f"🌐 Launching notebook: {notebook_url}")

try:
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False, args=['--start-maximized'])
        context = browser.new_context(viewport={'width': 1920, 'height': 1080})
        page = context.new_page()
        page.goto(notebook_url, wait_until='networkidle', timeout=30000)
        print("✓ Notebook opened in browser for debugging!")
        print("💡 Close browser window when done")
        while browser.connected:
            time.sleep(1)
        if browser.connected:
            browser.close()
except Exception as e:
    print(f"Error: {e}")
    import webbrowser
    webbrowser.open(notebook_url)
